Imports:

In [1]:
import numpy as np
from scipy.stats import pearsonr
from heapq import heappush, heappop, heappushpop
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import itertools
import pickle

Load the presaved channel data and word/category data:

In [2]:
all_data = np.load("all_data.npy") #holds all the data from channels
category_info = np.load("words_in_categories.npy") #category_info[cat][ptr] returns the number of the word(0...62) of the ptr'th word in the category cat
lengths = np.load("category_lengths.npy") #lengths[cat] is the number of words in category cat

Define some constants:

In [3]:
total_words = 63 

tStart = 0 #start time
tEnd = 650 #end time
tWidth = 100 #width of time slice
tIncr = 50 #increment in start time
tEx = 10 #number of examples to downsample to

training_amt = 8 #8 examples for training, 2 for testing
testing_amt = 10 - training_amt

np.random.seed(63)

Create and build TrainingData and TestingData matrices:

In [4]:
TrainingData = np.zeros((total_words,5,training_amt,256,650))#gives the pertinent data from all_data for the two categories
TestingData = np.zeros( (total_words,5,testing_amt,256,650)) #^
wordptr = -1 #the index of the current word, iterates from 0...total_words

for i in range(63):
    wordptr+=1

    excl = [-1]*10 #excl[j] = the j'th presentation number which should be saved for testing (e.g. excl[0] = 0 means the first presentation of the wordptr'th word should be saved for testing). Ignore -1's.
    
    for pres in range(testing_amt):
        while(1): #this loop repeatedly generates a random presentation until one which hasn't been reserved for testing has been found, and then breaks it
            nxtrand = np.random.randint(0,10)
            if(excl[nxtrand]==-1):
                excl[nxtrand]=nxtrand
                break
    for bandnum in range(5):
        ptr2 = 0 #points to which presentation(0...9) of wordptr'th word we are currently copying to TrainingData
        for pres in range(10):
            if(excl[pres]!=-1): #if reserved for testing, don't include in training data
                continue
           
            TrainingData[wordptr][bandnum][ptr2]=all_data[bandnum][i][pres] #sets the channel x time matrix for TrainingData[bandnum][wordptr][ptr2]
            ptr2+=1 #move to next presentation

    for bandnum in range(5): #this loop is same as above, except now we only want the testing presentations
        ptr2=0
        for pres in range(10):
            if(excl[pres]==-1):
                continue
            TestingData[wordptr][bandnum][ptr2] = all_data[bandnum][i][excl[pres]]
            ptr2+=1
            

Create vectors to hold best feature information based on Pearson Coefficients:

In [5]:
toSelect = 5 #number of top features to select

best_feature_vectors = np.zeros((total_words, training_amt,toSelect * tEx))
test_feature_vectors = np.zeros((total_words, testing_amt, toSelect * tEx))
timeSequences = np.zeros((total_words,5,12,training_amt,256,tEx))

Pick top feature vectors:

In [11]:
fixedc = int(tWidth/tEx)
ptrr = 0
for t in range(tStart, tEnd-tWidth+1, tIncr):
    ptrppp = 0
    for tEStart in range(t,t+tWidth-tEx+1,tEx):
        timeSequences[:,:,ptrr,:,:,ptrppp] = np.average(TrainingData[:,:,:,:,tEStart:tEStart+fixedc], axis = 4)
        ptrppp+=1
    ptrr+=1
print(str(timeSequences.shape))

for wordnum in range(total_words):
    SHheap = [] #heap of BTC + featurevector information used to select top 400
    
    for band_num in range(5): #frequency bands
        ptrr=0
        for t in range(tStart, tEnd-tWidth+1, tIncr): #various starts of time slice
            for channel in range(256): #eeg channels

                #pairwise correlations
                avg_p = 0
                avg_p2 = 0
                #print(str(wordnum) + " " + str(band_num) + " " + str(ptrr) + " " + str(channel))
                for i in range(training_amt-1):
                    for j in range(i+1,training_amt):
                        #if(wordnum == 1):
                       #     print(str(pearsonr(timeSequences[wordnum][band_num][ptrr][channel][i],timeSequences[wordnum][band_num][ptrr][channel][j])))
                        avg_p += pearsonr(timeSequences[wordnum][band_num][ptrr][i][channel],timeSequences[wordnum][band_num][ptrr][j][channel])[0]

                '''
                for word2 in range(total_words):
                    if(wordnum==word2):
                        continue
                    avg_p2 += pearsonr(AverageWord[wordnum][band_num][ptrr][channel], AverageWord[word2][band_num][ptrr][channel])[0]
                '''
                avg_p /= training_amt*(training_amt-1)/2 #want to maximize
                #avg_p2 /= (total_words-1) #want to minimize
                #ranking_measure = (avg_p - avg_p2)/2 #want to maximize
                if(len(SHheap)<400):
                    heappush(SHheap, (avg_p,band_num,t,channel, timeSequences[wordnum,band_num,ptrr,:,channel]))
                else:
                    heappushpop(SHheap, (avg_p,band_num,t,channel, timeSequences[wordnum,band_num,ptrr,:,channel]))
            ptrr+=1
    #pick top 5
    
    #f.write("Word " + str(wordnum) +"\n")
    print("Word " + str(wordnum))

    
    current_matrix = np.zeros( (training_amt,0))
    test_matrix = np.zeros( (testing_amt,0))
    
    for i in range(400):
        (avg_p,band_num,t,channel, timeSequenc) = heappop(SHheap)
        if(i>=400-toSelect):
            #this is da guy
            #f.write(str(400-i) + ". " + str(band_num) + "   " + str(t) + "   " + str(channel) + "   " + str(avg_p) + "\n")
            print(str(400-i) + ". " + str(band_num) + "   " + str(t) + "   " + str(channel) + "   " + str(avg_p))
            current_matrix = np.hstack( (current_matrix,timeSequenc))

            #construct testing matrix
            tmpo = np.zeros( (testing_amt,tEx))
            for itero in range(testing_amt):
                pp = 0
                for tEStart in range(t,t+tWidth-tEx+1,tEx):
                    tmpo[itero][pp] = np.average(TestingData[wordnum,band_num,itero,channel,tEStart:tEStart+int(tWidth/tEx)])
                    pp+=1
            test_matrix = np.hstack( (test_matrix,tmpo) )
            
    best_feature_vectors[wordnum] = current_matrix
    test_feature_vectors[wordnum] = test_matrix

(63, 5, 12, 8, 256, 10)
Word 0
5. 0   500   58   0.97414626929
4. 0   250   16   0.976603672895
3. 0   250   29   0.982799887577
2. 0   250   23   0.984346319517
1. 0   450   39   0.986853579328
Word 1
5. 0   350   247   0.922771480833
4. 0   450   31   0.927248448648
3. 0   400   247   0.936575436008
2. 0   300   73   0.947596205345
1. 0   200   239   0.950823540383
Word 2
5. 0   150   233   0.931991434803
4. 0   450   58   0.938864194273
3. 0   450   144   0.939616281831
2. 0   450   243   0.940000287145
1. 0   450   145   0.962295117817
Word 3
5. 0   200   38   0.984405804446
4. 0   200   46   0.984589580238
3. 0   200   28   0.986547647042
2. 0   200   29   0.987102458171
1. 0   400   35   0.989467676496
Word 4
5. 0   150   69   0.976214667503
4. 0   300   57   0.977821228133
3. 0   150   56   0.978149087461
2. 0   150   47   0.986665314791
1. 0   150   239   0.994502272076
Word 5
5. 0   300   35   0.960012413972
4. 0   100   190   0.962055774329
3. 0   100   171   0.968798208269
2

Word 48
5. 0   350   35   0.968103552335
4. 0   400   20   0.968583601181
3. 0   350   27   0.968891579628
2. 0   400   28   0.969389647754
1. 0   350   28   0.983240594961
Word 49
5. 0   200   45   0.98979890738
4. 0   350   45   0.990059237737
3. 0   200   37   0.992450898855
2. 0   150   45   0.995012933186
1. 0   150   37   0.996334526224
Word 50
5. 0   200   48   0.979865184758
4. 0   200   49   0.981283562034
3. 0   250   49   0.98306766454
2. 0   400   16   0.984656503826
1. 0   400   41   0.986023813352
Word 51
5. 0   350   47   0.96071671844
4. 0   400   47   0.963664076851
3. 0   350   54   0.96423368876
2. 0   400   48   0.969842259838
1. 0   400   40   0.971734931756
Word 52
5. 1   100   158   0.919936115518
4. 0   200   29   0.929608951417
3. 0   200   40   0.93598538443
2. 0   150   61   0.943716469402
1. 0   400   40   0.96072682107
Word 53
5. 0   0   206   0.958539334046
4. 0   0   196   0.960257650168
3. 0   0   183   0.966388761764
2. 0   0   195   0.972333079294
1. 0

Set-up for running the actual training and testing between every pair of categories: 

In [12]:
#dictionaries for storing all data
save_trainx = {}
save_trainy = {}
save_testx = {}
save_testy = {}

#initialize variable for overall average accuracy
avgacc = 0 

#initialize space to search for optimal C values
clist = np.logspace(-3,2,100)

Run training and testing (on separate data of course) for every combination of category pairs using a linear kernel SVM. Save and print results:

In [14]:
for cat1 in range(12):
    for cat2 in range(cat1+1,12):
        ptr = 0
        tot_words = int(lengths[cat1][0])+int(lengths[cat2][0])
        
        trainx = np.zeros ( ( training_amt * tot_words,toSelect * tEx))
        trainy = np.zeros( (training_amt * tot_words) )
        testx = np.zeros ( (testing_amt * tot_words, toSelect*tEx))
        testy = np.zeros( (testing_amt * tot_words))
        ptr2 = 0
        for itero in range(training_amt):
            while ptr<7 and category_info[cat1][ptr]!=-1:
                tword = category_info[cat1][ptr]
                
                trainx[ptr2] = best_feature_vectors[tword][itero]
                trainy[ptr2] = 0
                ptr2+=1
                
                ptr+=1
            ptr = 0
            while ptr < 7 and category_info[cat2][ptr]!=-1:
                tword = category_info[cat2][ptr]
                
                trainx[ptr2] = best_feature_vectors[tword][itero]
                trainy[ptr2] = 1
                ptr2+=1
                ptr+=1
        ptr = 0
        ptr2=0
        for itero in range(testing_amt):
            while ptr < 7 and category_info[cat1][ptr]!=-1:
                tword = category_info[cat1][ptr]
                testx[ptr2] = test_feature_vectors[tword][itero]
                testy[ptr2] = 0
                ptr2+=1
                ptr+=1
            ptr = 0
            while ptr < 7 and category_info[cat2][ptr]!=-1:
                tword = category_info[cat2][ptr]

                testx[ptr2] = test_feature_vectors[tword][itero]
                testy[ptr2] = 1
                ptr2+=1
                ptr+=1
            ptr = 0
        save_trainx[(cat1,cat2)] = trainx
        save_trainy[(cat1,cat2)] = trainy
        save_testx[(cat1,cat2)] = testx
        save_testy[(cat1,cat2)] = testy

        #run kfold cross validation for parameters
        bst_acc = 0
        bst_c = 0
        
        for c in clist:
                avg_acc = 0
                for fold in range(4):
                    fold_sz = int(training_amt*tot_words/4)
                    valid_x = trainx[(fold_sz*fold):((fold_sz)*(fold+1))]
                    valid_y = trainy[(fold_sz*fold):((fold_sz)*(fold+1))]
                    tr_x = np.concatenate( (trainx[0:(fold_sz*fold)],trainx[(fold+1)*fold_sz:(training_amt*tot_words)]), axis = 0)
                    tr_y = np.concatenate( (trainy[0:(fold_sz*fold)],trainy[(fold+1)*fold_sz:(training_amt*tot_words)]), axis = 0)

                    scaler = StandardScaler()
                    tr_x = scaler.fit_transform(tr_x)
                    valid_x = scaler.transform(valid_x)
                    tr_y = np.ravel(tr_y)
                    valid_y = np.ravel(valid_y)

                    classifier = LinearSVC(C=c)
                    classifier.fit(tr_x,tr_y)
                    avg_acc += (classifier.score(valid_x,valid_y) )/4.0
                if(avg_acc > bst_acc):
                    bst_acc = avg_acc
                    bst_c = c
                    
        print("For " + str(cat1) + " and " + str(cat2) + " we picked C = " + str(bst_c))
        
        #create final classifier with best C value
        classifier = LinearSVC(C=bst_c)
        
        #adjust data to the classifier
        scaler = StandardScaler()
        trainx = scaler.fit_transform(trainx)
        testx = scaler.transform(testx)
        trainy = np.ravel(trainy)
        testy = np.ravel(testy)
        
        #fit training data to classifier
        classifier.fit(trainx, trainy)
        #test on testing data
        myscore = classifier.score(testx,testy)
        avgacc+=myscore
        print("Has accuracy " + str(myscore))
        print("======")

pickle.dump( (save_trainx,save_trainy,save_testx,save_testy), open("created_data.p","wb") )
avgacc/=(12*11/2)
print("Average score was " + str(avgacc))

print("Finished!")

For 0 and 1 we picked C = 8.69749002618
Has accuracy 0.636363636364
For 0 and 2 we picked C = 2.42012826479
Has accuracy 0.730769230769
For 0 and 3 we picked C = 35.1119173422
Has accuracy 0.636363636364
For 0 and 4 we picked C = 44.3062145758
Has accuracy 0.727272727273
For 0 and 5 we picked C = 12.3284673944
Has accuracy 0.590909090909
For 0 and 6 we picked C = 0.16681005372
Has accuracy 0.590909090909
For 0 and 7 we picked C = 10.9749876549
Has accuracy 0.636363636364
For 0 and 8 we picked C = 89.0215085445
Has accuracy 0.636363636364
For 0 and 9 we picked C = 8.69749002618
Has accuracy 0.636363636364
For 0 and 10 we picked C = 0.533669923121
Has accuracy 0.818181818182
For 0 and 11 we picked C = 5.46227721768
Has accuracy 0.772727272727
For 1 and 2 we picked C = 0.599484250319
Has accuracy 0.5
For 1 and 3 we picked C = 49.7702356433
Has accuracy 0.6
For 1 and 4 we picked C = 39.4420605944
Has accuracy 0.4
For 1 and 5 we picked C = 15.5567614393
Has accuracy 0.75
For 1 and 6 we pick

=========================================================

Final Statistics: 

Average score:
    0.65457192983

Best pair score: 
    0.875 with categories 2 and 11 and C = 89.0215085445

=========================================================